In [1]:
from fastai.vision.all import *
from models.VAE import VariationalAutoEncoder
from torchsummary import summary
from functools import partial
from util.utils import rmse_loss, vae_loss

In [2]:
path = Path('data/mnist_png')
if not path.exists():
    path = untar_data(URLs.MNIST, dest='data')

In [3]:
Path.BASE_PATH = path

In [ ]:
path.ls()

In [4]:
datablock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW), ImageBlock(cls=PILImageBW)),
    get_items=get_image_files,
    splitter=RandomSplitter(seed=42)
)

In [5]:
dataloaders = datablock.dataloaders(path)

In [6]:
model = VariationalAutoEncoder(
    input_dim=[1, 28, 28],
    conv_filters=[32, 64, 64, 64],
    conv_t_filters=[64, 64, 32, 1],
    kernel_sizes=[3, 3, 3, 3],
    strides=[1, 2, 2, 1],
    z_dim=2
).to(device='cuda')

In [ ]:
summary(model.encoder, (1, 28, 28))

In [7]:
learner = Learner(
    dls=dataloaders,
    model=model,
    loss_func=partial(vae_loss, model=model)
)

In [8]:
learner.fit(10, 2e-6)

epoch,train_loss,valid_loss,time
0,444.416748,441.750397,00:39
1,392.554047,389.473328,00:33
2,340.942047,338.529236,00:33
3,310.618805,309.493103,00:33
4,293.771912,292.356537,00:33
5,280.839813,279.580200,00:33
6,272.602936,271.201752,00:34
7,268.004608,266.324402,00:35
8,264.888855,263.712128,00:36
9,262.911469,262.513397,00:34
